In [19]:
#Importing Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import ConvLSTM2D
from keras.layers import Flatten
from numpy import array

In [20]:
#Import Raw Data and convert it to CSV 

url = "https://raw.githubusercontent.com/rohitash-chandra/CMTL_dynamictimeseries/master/IndianOcean/rawtrain1985-2001.txt"
df = pd.read_csv(url, sep = "\t", header = None)
df.columns = ['id','date','longitude','latitude','speed']
df = df.drop(['date'], axis = 1)
df.to_csv('adjusted.csv')


In [21]:
# Using a Univariate LSTM, therefore only working with Speed 

# Extracting speed data from Dataset 

speed = array(df['speed'])
speed = speed.reshape((len(speed), 1))
speed

array([[20],
       [20],
       [25],
       ...,
       [35],
       [30],
       [25]], dtype=int64)

In [22]:
# Splitting the Sequence To Prepare the Training and Test Data. Data Preprocessing. 

def split_seq(timeseries_data, n_steps):
    X, y =[],[]
    for i in range(len(timeseries_data)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(timeseries_data)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [23]:
# Defining RMSE Function to test Performance

def rmse(pred, actual):
    temp = np.sqrt(((pred-actual)**2).mean())
    return temp

In [24]:
# Initialising Features of LSTM 

n_steps=4
n_features = 1



In [25]:
# From Speed DataFrame, Separate and Prepare (Split and make 3D) Test and Train Data 

train = speed[0:9000]
test = speed[9000:9364]

x_train, y_train = split_seq(train, n_steps)
x_test, y_test = split_seq(test, n_steps)


x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
print(x_train.shape)

x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features))
print(x_test.shape)

y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
print(y_train.shape)

y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
print(y_test.shape)

(8996, 4, 1)
(360, 4, 1)
(8996, 1, 1)
(360, 1, 1)


In [26]:
# Defining and Fitting the Conv LSTM Network 

#Define the Model
model=Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

#Fit the Model 
model.fit(x_train,y_train,epochs=100,verbose=0)

#Predict with the Fitted Model 
y_train_predicted=model.predict(x_train)
y_test_predicted=model.predict(x_test)

#Check RMSE
train_acc=rmse(y_train_predicted,y_train)
test_acc=rmse(y_test_predicted,y_test)


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [28]:
#Displaying Train and Test Accuracies 

print(train_acc, 'is the RMSE for the Train Data') 
print(test_acc, 'is the RMSE for the Test Data')

36.538490402665985 is the RMSE for the Train Data
36.9689885445247 is the RMSE for the Test Data
